In [1]:
import os
from lxml import etree 
doc = etree.parse('../Data/xml_data/provenance.xml')

In [2]:
provenance = doc.findall('recordList/record/provenance/provenance.project.txt')
for item in provenance:
    if item.text == None:
        provenance.remove(item)
    else:
        continue

In [70]:
bydescent_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if ' transcribed in Salverda de Grave 1918, p. 96.}' in elem:
            continue
        if 'by descent' in elem and not 'estate' in elem:
            bydescent_items.append(elem)
        else:
            continue

In [67]:
widow_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'his widow' in elem and not 'estate' in elem:
            widow_items.append(elem)
        elif 'her widow' in elem and not 'estate' in elem:
            widow_items.append(elem)
        else:
            continue

In [68]:
son_daughter_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'his son' in elem and not 'estate' in elem:
            son_daughter_items.append(elem)
        if 'her son' in elem and not 'estate' in elem:
            son_daughter_items.append(elem)
        if 'his daughter' in elem and not 'estate' in elem:
            son_daughter_items.append(elem)
        if 'her daughter' in elem and not 'estate' in elem:
            son_daughter_items.append(elem)

In [30]:
niece_nephew_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'his niece' in elem and not 'estate' in elem:
            niece_nephew_items.append(elem)
        if 'her niece' in elem and not 'estate' in elem:
            niece_nephew_items.append(elem)
        if 'his nephew' in elem and not 'estate' in elem:
            niece_nephew_items.append(elem)
        if 'her nephew' in elem and not 'estate' in elem:
            niece_nephew_items.append(elem)

In [7]:
grand_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if ' grand' in elem and not 'estate' in elem:
            grand_items.append(elem)

In [90]:
#by descent items
for item in bydescent_items:
    test_item = item
    if '?' in item and not '?-' in item and not '-?' in item:
        proper_format = '? by descent'
        readable_sent = 'Possibly inherited by descent'
    else:
        proper_format = 'by descent'
        readable_sent = 'Inherited by descent'
    through = False
    through_name = ''
    family_relation = ''
    to_name = ''
    to_extra = ''
    date = ''
    place = ''
    source = ''
    extra = ''
    to_name_item =''
    place_item = ''
    if item.startswith('{') or item.startswith(' {'):
        curly_split = item.split('}')
        source = curly_split[0].replace('{', '').strip()
        source = '{'+source+'}'
    else:
        curly_split = item.split('{')
        if len(curly_split) >1:
            source = curly_split[1].replace('}','')
            source = '{'+source+'}'
    through_split = item.split(' through ')
    if len(through_split) != 1:
        through = True
        to_split = through_split[1].split(' to ')
        through_name = to_split[0].strip()
    test_split = test_item.replace(source,'')
    to_split = test_split.split(' to ')
    if len(to_split) >1:
        comma_split = to_split[1].split(',')
        if '1' not in comma_split[-1] or '2' not in comma_split[-1]:
            place = comma_split[-1].strip()
        for elem in comma_split:
            if len(comma_split) >1:
                if 'his ' in elem or 'her ' in elem:
                    family_relation = elem.strip()
                if family_relation != '':
                    to_name = comma_split[1].split('(')
                    to_name = to_name[0].strip()
                    if len(comma_split) >2:
                        if '(' in comma_split[2]:
                            to_extra = comma_split[2].split('(')
                            to_extra = to_extra[0].strip()
                if family_relation == '': 
                    to_name = comma_split[0].split('(')
                    to_name = to_name[0].strip()
                    if '(' in comma_split[1]:
                        to_extra = comma_split[1].split('(')
                        to_extra = to_extra[0].strip()
            if len(comma_split) == 1:
                to_split = comma_split[0].split(' to ')
                to_name = to_split[0].split('(')
                to_name = to_name[0].strip()
    para_split = item.split('(')
    for item in para_split:
        if ')' in item:
            comma_split = item.split(',')
            for item in comma_split:
                if item.replace('-', '').replace(')','').isnumeric():
                    date = item.replace(')', '').strip()
    if to_name in place:
        place = ''
    #see if there is extra information missed
    test = test_item.replace(source,'')
    if ',' in test and place != '' and to_name !='':
        items = test.split(',')
        if len(items) > 1:
            for item in items:
                if to_name in item:
                    to_name_item = item
                if place in item:
                    place_item = item
            if (items.index(to_name_item)-items.index(place_item)) != -1:
                index = (items.index(to_name_item) +1)
                extra = items[index]
                extra = extra.strip()
                if items.index(to_name_item) +2 != items.index(place_item):
                    index2 = (items.index(to_name_item) +2)
                    extra = extra+','+items[index2]
                    extra = extra.strip()
                    if items.index(to_name_item) +3 != items.index(place_item):
                        index3 = (items.index(to_name_item) +3)
                        extra = extra+','+items[index3]
                        extra = extra.strip()
    if date in extra or to_name in extra:
        extra = ''
    if date in to_extra:
        to_extra = to_extra.replace(date,'')
    #form proper formatted provenance sentence
    if test_item.startswith(source):
        proper_format = f"{source} {proper_format}"
    if through:
        proper_format = f"{proper_format} through {through_name}"
    if family_relation != '':
        proper_format = f"{proper_format} to {family_relation},"
    if to_name != '' and family_relation == '':
        proper_format = f"{proper_format} to {to_name}"
    if to_name != '' and family_relation != '':
        proper_format = f"{proper_format} {to_name}"
    if to_extra != '':
        proper_format = f"{proper_format}, {to_extra}"
    if date != '':
        proper_format = f"{proper_format} ({date})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if source != '' and not test_item.startswith(source):
        proper_format = f"{proper_format} {source}"
    #print('Proper format:',proper_format)
    #form readable provenance sentence
    if through:
        readable_sent = f"{readable_sent} through {through_name}"
    if family_relation != '':
        readable_sent = f"{readable_sent} by {family_relation},"
    if to_name != '' and family_relation == '':
        readable_sent =f"{readable_sent} by {to_name}"
    if to_name != '' and family_relation != '':
        readable_sent = f"{readable_sent}, {to_name}"
    if to_extra != '':
        readable_sent = f"{readable_sent}, {to_extra}"
    if date != '':
        readable_sent = f"{readable_sent} ({date})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    readable_sent = f"{readable_sent}."
    #print('Readable sentence:', readable_sent)
    #print('\n')
    
    #check if a mistake was made
    if proper_format.strip() != test_item.strip():
        print('Now:',test_item)
        print('Proper_format:', proper_format, '\n')

Now:  ? by descent  to his grandson,  Leonard Pieter Joseph Michielsen (1872-1944), The Hague
Proper_format:  ? by descent to his grandson, Leonard Pieter Joseph Michielsen (1872-1944), The Hague 

Now:  by descent hrough the family to Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen
Proper_format:  by descent to Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen 

Now:  by descent throught the family to Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen
Proper_format:  by descent to Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen 

Now:  ? by descent through the family of the sitter to  Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch)
Proper_format:  ? by descent through the family of the sitter to Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Sant

In [ ]:
#Mistakes: through often not spelled correctly, sometimes forgotten commas or extra spaces

In [64]:
#other items
for item in son_daughter_items[21:41]:
    test_item = item
    source = ''
    family_relation = ''
    name = ''
    name_extra = ''
    date = ''
    place = ''
    extra = ''
    to_name_item =''
    place_item = ''
    source_start = ''
    #certain or not?
    if '?' in item:
        proper_format = '?'
        readable_sent = 'Possibly inherited by'
    else:
        proper_format = ''
        readable_sent = 'Inherited by'
    #source
    if '}' in item or '{' in item:
        if item.startswith('{') or item.startswith(' {'):
            curly_split = item.split('}')
            source = curly_split[0].replace('{', '').strip()
            source = '{'+source+'}'
            source_start = source.strip()
        else:
            curly_split = item.split('{')
            if len(curly_split) >1:
                source = curly_split[1].replace('}','').strip()
                source = '{'+source+'}'
            curly_split2 = item.split('}')
            if ' his' in curly_split2[1] or ' her' in curly_split2[1]:
                source = curly_split2[0].strip()
    item = item.replace(source, '')
    #family relation
    comma_split = item.split(',')
    #if it starts with a source
    if '}' in comma_split[0]:
        source_split = comma_split[0].split('}')
        family_relation = source_split[1].replace('?', '').strip()
    else:
        family_relation = comma_split[0].replace('?', '').strip()
    #name and extra info on name
    name = comma_split[1].strip()
    if '(' in name:
        name_split = name.split('(')
        name = name_split[0].strip()
    if '(' not in name:
        name_extra = comma_split[2]
        name_extra = name_extra.split('(')
        name_extra = name_extra[0].strip()
    #date
    para_split = test_item.split('(')
    for items in para_split:
        if ')' in items:
            item = items.split(')')
            if '1' in item[0] or '2' in item[0]:
                date = item[0].strip()
    #place
    place = comma_split[-1].strip()
    if date in place:
        place = ''
    if '{' in place:
        place = place.split('{')
        place = place[0].strip()
    if name_extra in place:
        name_extra = ''
    #see if there is extra information missed
    test = test_item.replace(source,'')
    if ',' in test and place != '' and name !='':
        items = test.split(',')
        if len(items) > 1:
            for item in items:
                if name in item:
                    to_name_item = item
                if place in item:
                    place_item = item
            if (items.index(to_name_item)-items.index(place_item)) != -1:
                index = (items.index(to_name_item) +1)
                extra = items[index]
                extra = extra.strip()
                if items.index(to_name_item) +2 != items.index(place_item):
                    index2 = (items.index(to_name_item) +2)
                    extra = extra+', '+items[index2]
                    extra = extra.strip()
    if date in extra or to_name in extra:
        extra = ''
    #forming the sentences
    #form proper formatted provenance sentence
    if source_start != '':
        proper_format = f"{source_start}{proper_format}"
    if family_relation != '':
        proper_format = f"{proper_format} {family_relation},"
    if name != '':
        proper_format = f"{proper_format} {name}"
    if name_extra != '':
        proper_format = f"{proper_format}, {name_extra}"
    if date != '':
        proper_format = f"{proper_format} ({date})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if source != '' and source_start == '':
        proper_format = f"{proper_format} {source}"
    #print('Proper format:',proper_format)
    #form readable provenance sentence
    if family_relation != '':
        readable_sent = f"{readable_sent} {family_relation},"
    if name != '':
        readable_sent = f"{readable_sent} {name}"
    if name_extra != '':
        readable_sent = f"{readable_sent}, {name_extra}"
    if date != '':
        readable_sent = f"{readable_sent} ({date})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    readable_sent = f"{readable_sent}."
   #print('Readable sentence:', readable_sent)
   # print('\n')
    
    #check if a mistake was made
    if proper_format.strip() != test_item.strip():
        print(test_item)
        print('Proper_format:', proper_format, '\n')

{Moes I, 1897, p. 428, no. 3559.} his son or grandson,  Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch)
Proper_format: {Moes I, 1897, p. 428, no. 3559.} his son or grandson, Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch) 

{Moes II, 1905, p. 42, no. 4667.} his son or grandson,  Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch)
Proper_format: {Moes II, 1905, p. 42, no. 4667.} his son or grandson, Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch) 

{Moes II, 1905, pp. 632-633, no. 9173.} his son or grandson,  Jonkheer Ernest Louis Willy Marc Hoeufft, Lord of Velsen and Santpoort (1903-65), Meeuwen (near ’s-Hertogenbosch)
Proper_format: {Moes II, 1905, pp. 632-633, no. 9173.} his son or grandson, Jonkheer Ernest Louis Willy Marc Hoe

In [ ]:
#again, mostly extra spaces is where it's going wrong